### Retrieve and save data

In [1]:
from codesim.problem import StraightLine 

gen = StraightLine(n_ops=20, 
                   n_vars=3,
                   n_instances=2)

# Generate and save 3 batches of 30 problems each
for i in range(3):
    gen.generate_data(n_programs=30)
    gen.to_file(suffix="batch-" + str(i+1))
    gen.reset()


In [2]:
from codesim.problem import CriticalPath 

gen = CriticalPath(n_ops=20, 
                   n_vars=4,
                   len_critical_path=10)

# Generate and save 3 batches of 30 problems each
for i in range(3):
    gen.generate_data(n_programs=30)
    gen.to_file(suffix="batch-" + str(i+1))
    gen.reset()

In [3]:
from codesim.problem import ParallelPaths 

gen = ParallelPaths(n_ops=20, 
                    n_vars=3,
                    n_instances=2)

# Generate and save 3 batches of 30 problems each
for i in range(3):
    gen.generate_data(n_programs=30)
    gen.to_file(suffix="batch-" + str(i+1))
    gen.reset()

In [4]:
from codesim.problem import Loops 

gen = Loops(n_loops=5, 
            n_noisy_loops=2,
            min_loop_length=3,
            max_loop_length=5)

# Generate and save 3 batches of 30 problems each
for i in range(3):
    gen.generate_data(n_programs=30)
    gen.to_file(suffix="batch-" + str(i+1))
    gen.reset()

In [5]:
from codesim.problem import Sort 

gen = Sort(n_vars=10,
           ascending=True)

# Generate and save 3 batches of 30 problems each
for i in range(3):
    gen.generate_data(n_programs=30)
    gen.to_file(suffix="batch-" + str(i+1))
    gen.reset()

In [ ]:
import sys
from codesim.kim_schuster import real_main
%load_ext autoreload
%autoreload 2
# simulate CLI run
sys.argv = ["codesim/kim_schuster.py", 
            "--max_items_per_box", "3",
            "--num_samples", "30"]

real_main()

### Generate prompts

In [1]:
# Test the prompts
import json 
from codesim.prompt import StraightLine as SL

# Get the data
data = json.load(open("./data/StraightLine/n_ops-20_n_vars-3_n_instances-2_batch-1.json"))
p_syn, p_nat, lab_syn, lab_nat = data[0].values()

# Syn
var_name = list(lab_syn.keys())[0]  # can sample here
gt = lab_syn[var_name]
question = SL.questions_syn[0].format(varname=var_name)
prompt_syn = SL.user_cot.format(prefix="Here's some code:", 
                                problem=p_syn, 
                                question=question)
print(prompt_syn)

print("="*30)
# Nat
agent_name = list(lab_nat.keys())[0]  # can sample here
object_name = list(lab_nat[agent_name].keys())[0]  # same...
gt = lab_nat[agent_name][object_name]
question = SL.questions_nat[0].format(varname=object_name, agentname=agent_name)
prompt_nat = SL.user_cot.format(prefix="",
                                problem=p_nat,
                                question=question)
print(prompt_nat)

Here's some code: 
a0=4; a1=5
b0=2; b1=2
c0=7; c1=3
b0 += 7
b1 -= 1
a1 -= 1
c0 += a0
a0 -= a0
c1 += 5
a1 += 7
c0 += b0
b0 -= b0
a1 += 5
b1 += 5
c0 += 4
c1 -= 2
a0 += 1
c0 -= 1
c0 -= 14
a1 -= 2
b0 += 4
a1 -= 9
b1 -= 2
c1 += 6
b0 += 1
c0 -= 1
b0 += 10

What is the value of a0 at the end of the computation?
Think step by step and provide the answer between <answer></answer> tags.
For example, reply with <answer>1</answer> if the answer is 1.

 
There are 3 agents, ['a', 'b', 'c']. Each of them has 2 items, ['obj-0', 'obj-1'].
Here is the initial quantity of each object per agent.

Agent-a has 4 obj-0, 5 obj-1.
Agent-b has 2 obj-0, 2 obj-1.
Agent-c has 7 obj-0, 3 obj-1.

Here's the list of potential interactions between agents.
An agent can give a quantity of one their objects to another agent. 
In that case, they lose that quantity of that object and the other agent increases theirs'.
An agent can lose some or all of their objects. In that case, they lose that quantity of that object.
An 